In [1]:
!pip install -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 17.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━

##Train custom chat-model on custom data

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import OpenAIEmbeddings
import pickle




import os
os.environ["OPENAI_API_KEY"]=""
print("Loading data...")
loader = UnstructuredFileLoader("state_of_the_union.txt")
raw_documents = loader.load()


print("Splitting text...")
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=600,
    chunk_overlap=100,
    length_function=len,
)
documents = text_splitter.split_documents(raw_documents)

DB_FAISS_PATH = "vectorstore/db_faiss"
print("Creating vectorstore...")
embeddings = OpenAIEmbeddings()
print("sommmmmmmmmmmmm")
vectorstore = FAISS.from_documents(documents, embeddings)
# with open("vectorstore.pkl", "wb") as f:
#     pickle.dump(vectorstore, f)

vectorstore.save_local(DB_FAISS_PATH)
print("done")

Loading data...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Splitting text...
Creating vectorstore...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


sommmmmmmmmmmmm
done


Load the model and evaluate

In [37]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
import pickle
import sys

In [32]:
def load_retriever():
    # with open("/content/vectorstore/db_faiss/index.pkl", "rb") as f:
    #     vectorstore = pickle.load(f)
    retriever = FAISS.load_local("/content/vectorstore/db_faiss", embeddings)
    return retriever
def get_basic_qa_chain():
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    retriever = load_retriever()
    memory = ConversationBufferMemory(
        memory_key="chat_history", return_messages=True)
    model = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever.as_retriever(),verbose=False,
        memory=memory)
    return model


In [33]:
chain=get_basic_qa_chain()
output = chain({"question":"What did the president say about Ketanji Brown Jackson"})
print(output['answer'])

The President mentioned that Ketanji Brown Jackson is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. He also highlighted her background as a former top litigator in private practice and a former federal public defender, coming from a family of public school educators and police officers. Additionally, he mentioned that since her nomination, she has received broad support from various groups, including the Fraternal Order of Police and former judges appointed by Democrats and Republicans.


In [34]:
output = chain({"question":"What was his stance on Ukraine?"})
print(output['answer'])

The President's stance on Ukraine is one of strong support for the Ukrainian people in their fight for freedom. He emphasized that the United States stands with Ukraine and is providing military, economic, and humanitarian assistance to aid them in defending their country. The President also made it clear that U.S. forces are not engaged in conflict with Russian forces in Ukraine.


In [35]:
output = chain({"question":"Did he mention Stephen Breyer?"})
print(output['answer'])

Yes, the President did mention Justice Stephen Breyer in the context provided.


Continues Chating on data

In [38]:
while True:
  query=input(f"question: ")
  if query == 'exit':
    print('Exiting')
    break
  if query =='':
    continue
  result=chain({'question':query})
  print(f"Answer: " +result["answer"])

question: Did he mention Stephen Breyer?
Answer: Yes, the President did mention Justice Stephen Breyer in his remarks.
question: What was his stance on Ukraine?
Answer: The President's stance on Ukraine is one of strong support for the Ukrainian people in their fight for freedom. He emphasized that the United States stands with Ukraine and is providing military, economic, and humanitarian assistance to aid them in defending their country. The President also made it clear that U.S. forces are not engaged in conflict with Russian forces in Ukraine. Additionally, he mentioned releasing oil reserves to help with gas prices at home and expressed confidence that the world will emerge stronger from Putin's war on Ukraine.
question: What did the president say about Ketanji Brown Jackson
Answer: The President mentioned that Ketanji Brown Jackson is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. He highlighted her background as a former top litigator

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
